Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

/home/pablo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
beta = 0.01


graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    hidden_layer_size = 1024

    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    hidden_biases = tf.Variable(tf.zeros([hidden_layer_size]))
    output_weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    output_bias = tf.Variable(tf.zeros([num_labels]))


    def myNetwork(input_data):
        # Hidden layer
        activations_hidden = tf.matmul(input_data, hidden_weights) + hidden_biases
        hidden_y = tf.nn.relu(activations_hidden)

        # Output layer
        output_activations = tf.matmul(hidden_y, output_weights) + output_bias
        output = tf.nn.softmax(output_activations)
       
        return output

    # Training computation.
    output_prediction = myNetwork(tf_train_dataset)
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_prediction)
    regularizer = tf.nn.l2_loss (hidden_weights) + tf.nn.l2_loss (output_weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.

    valid_prediction = myNetwork(tf_valid_dataset)
    test_prediction = myNetwork(tf_test_dataset)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, output_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3152.890137
Minibatch accuracy: 13.3%
Validation accuracy: 16.0%
Minibatch loss at step 500: 22.753445
Minibatch accuracy: 76.6%
Validation accuracy: 74.5%
Minibatch loss at step 1000: 1.986778
Minibatch accuracy: 77.3%
Validation accuracy: 74.0%
Minibatch loss at step 1500: 1.842022
Minibatch accuracy: 75.8%
Validation accuracy: 74.2%
Minibatch loss at step 2000: 1.898226
Minibatch accuracy: 71.1%
Validation accuracy: 74.0%
Minibatch loss at step 2500: 1.850006
Minibatch accuracy: 72.7%
Validation accuracy: 74.3%
Minibatch loss at step 3000: 1.929944
Minibatch accuracy: 69.5%
Validation accuracy: 73.9%
Test accuracy: 79.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % 2
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, output_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3145.215820
Minibatch accuracy: 7.8%
Validation accuracy: 10.6%
Minibatch loss at step 500: 22.488228
Minibatch accuracy: 93.0%
Validation accuracy: 63.0%
Minibatch loss at step 1000: 1.791135
Minibatch accuracy: 100.0%
Validation accuracy: 71.6%
Minibatch loss at step 1500: 1.644892
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 2000: 1.639773
Minibatch accuracy: 100.0%
Validation accuracy: 71.4%
Minibatch loss at step 2500: 1.637345
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 3000: 1.635680
Minibatch accuracy: 100.0%
Validation accuracy: 71.3%
Test accuracy: 77.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [49]:
batch_size = 128
beta = 0.01


graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    hidden_layer_size = 1024

    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    hidden_biases = tf.Variable(tf.zeros([hidden_layer_size]))
    output_weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    output_bias = tf.Variable(tf.zeros([num_labels]))

    rate = tf.placeholder("float")

    def myNetwork(input_data):
        # Hidden layer
        activations_hidden = tf.matmul(input_data, hidden_weights) + hidden_biases
        hidden_y = tf.nn.relu(activations_hidden)

        # Output layer
        output_activations = tf.matmul(hidden_y, output_weights) + output_bias
        output = tf.nn.softmax(output_activations)
       
        return output
    
    def myNetwork_t(input_data):
        # Hidden layer
        activations_hidden = tf.matmul(input_data, hidden_weights) + hidden_biases
        hidden_y = tf.nn.relu(activations_hidden)
        hidden_y = tf.nn.dropout(hidden_y, rate)
        
        # Output layer
        output_activations = tf.matmul(hidden_y, output_weights) + output_bias
        output = tf.nn.softmax(output_activations)
        
        return output
    
    # Training computation.
    output_prediction = myNetwork_t(tf_train_dataset)
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_prediction)
    regularizer = tf.nn.l2_loss (hidden_weights) + tf.nn.l2_loss (output_weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.

    valid_prediction = myNetwork(tf_valid_dataset)
    test_prediction = myNetwork(tf_test_dataset)

In [50]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % 2
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, rate : 0.5}
        _, l, predictions = session.run([optimizer, loss, output_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3146.071533
Minibatch accuracy: 11.7%
Validation accuracy: 14.5%
Minibatch loss at step 500: 22.462399
Minibatch accuracy: 94.5%
Validation accuracy: 70.0%
Minibatch loss at step 1000: 1.805417
Minibatch accuracy: 97.7%
Validation accuracy: 71.0%
Minibatch loss at step 1500: 1.662773
Minibatch accuracy: 97.7%
Validation accuracy: 71.2%
Minibatch loss at step 2000: 1.655289
Minibatch accuracy: 97.7%
Validation accuracy: 71.2%
Minibatch loss at step 2500: 1.656005
Minibatch accuracy: 97.7%
Validation accuracy: 71.2%
Minibatch loss at step 3000: 1.648754
Minibatch accuracy: 98.4%
Validation accuracy: 71.1%
Test accuracy: 77.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [56]:
batch_size = 128
beta = 0.005

hidden_size_1 = 1024
hidden_size_2 = int(hidden_size_1/2)
hidden_size_3 = int(hidden_size_2/2)
hidden_size_4 = int(hidden_size_3/2)

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.

    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    rate = tf.placeholder("float")
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    hidden_weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size_1]))
    hidden_biases_1 = tf.Variable(tf.zeros([hidden_size_1]))

    hidden_weights_2 = tf.Variable(tf.truncated_normal([hidden_size_1,hidden_size_2]))
    hidden_biases_2 = tf.Variable(tf.zeros([hidden_size_2]))

    hidden_weights_3 = tf.Variable(tf.truncated_normal([hidden_size_2,hidden_size_3]))
    hidden_biases_3 = tf.Variable(tf.zeros([hidden_size_3]))
    
    hidden_weights_4 = tf.Variable(tf.truncated_normal([hidden_size_3,hidden_size_4]))
    hidden_biases_4 = tf.Variable(tf.zeros([hidden_size_4]))
    
    output_weights = tf.Variable(tf.truncated_normal([hidden_size_4, num_labels]))
    output_bias = tf.Variable(tf.zeros([num_labels]))

    
    def myNetwork_t(input_data):
        # Hidden layer1
        activations_hidden_1 = tf.matmul(input_data, hidden_weights_1) + hidden_biases_1
        hidden_1 = tf.nn.relu(activations_hidden_1)
        hidden_1 = tf.nn.dropout(hidden_1, rate)

        # Hidden layer2
        activations_hidden_2 = tf.matmul(hidden_1, hidden_weights_2) + hidden_biases_2
        hidden_2 = tf.nn.relu(activations_hidden_2)
        hidden_2 = tf.nn.dropout(hidden_2, rate)

        # Hidden layer3
        activations_hidden_3 = tf.matmul(hidden_2, hidden_weights_3) + hidden_biases_3
        hidden_3 = tf.nn.relu(activations_hidden_3)
        hidden_3 = tf.nn.dropout(hidden_3, rate)

        # Hidden layer4
        activations_hidden_4 = tf.matmul(hidden_3, hidden_weights_4) + hidden_biases_4
        hidden_4 = tf.nn.relu(activations_hidden_4)
        hidden_4 = tf.nn.dropout(hidden_4, rate)
        
        # Output layer
        output_activations = tf.matmul(hidden_4, output_weights) + output_bias
        output = tf.nn.softmax(output_activations)
        
        return output
    
    
    
    def myNetwork(input_data):
        # Hidden layer1
        activations_hidden_1 = tf.matmul(input_data, hidden_weights_1) + hidden_biases_1
        hidden_1 = tf.nn.relu(activations_hidden_1)

        # Hidden layer2
        activations_hidden_2 = tf.matmul(hidden_1, hidden_weights_2) + hidden_biases_2
        hidden_2 = tf.nn.relu(activations_hidden_2)

        # Hidden layer3
        activations_hidden_3 = tf.matmul(hidden_2, hidden_weights_3) + hidden_biases_3
        hidden_3 = tf.nn.relu(activations_hidden_3)

        # Hidden layer4
        activations_hidden_4 = tf.matmul(hidden_3, hidden_weights_4) + hidden_biases_4
        hidden_4 = tf.nn.relu(activations_hidden_4)
        
        # Output layer
        output_activations = tf.matmul(hidden_4, output_weights) + output_bias
        output = tf.nn.softmax(output_activations)
        
        return output
    
    # Training computation.
    output_prediction = myNetwork_t(tf_train_dataset)
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_prediction)
    regularizer = tf.nn.l2_loss (hidden_weights_1) + tf.nn.l2_loss (hidden_weights_2) + tf.nn.l2_loss (hidden_weights_3) + tf.nn.l2_loss (hidden_weights_4) + tf.nn.l2_loss (output_weights)
    loss = tf.reduce_mean(loss + beta * regularizer)

    # Optimizer
    # optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    
    # Predictions for the training, validation, and test data.

    valid_prediction = myNetwork(tf_valid_dataset)
    test_prediction = myNetwork(tf_test_dataset)
    


In [59]:
num_steps = 9001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, rate : 0.5}
        _, l, predictions = session.run([optimizer, loss, output_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2889.661377
Minibatch accuracy: 7.8%
Validation accuracy: 11.9%
Minibatch loss at step 500: 242.493179
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 22.018501
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 3.491953
Minibatch accuracy: 71.1%
Validation accuracy: 72.6%
Minibatch loss at step 2000: 2.017633
Minibatch accuracy: 74.2%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 1.837902
Minibatch accuracy: 82.0%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 1.884557
Minibatch accuracy: 74.2%
Validation accuracy: 82.1%
Minibatch loss at step 3500: 1.893664
Minibatch accuracy: 71.9%
Validation accuracy: 81.9%
Minibatch loss at step 4000: 1.816578
Minibatch accuracy: 81.2%
Validation accuracy: 82.1%
Minibatch loss at step 4500: 1.825344
Minibatch accuracy: 80.5%
Validation accuracy: 82.6%
Minibatch loss at step 5000: 1.774624
Minibatch accuracy: 86.7%
Validation acc